In [1]:
from skimage.morphology import skeletonize
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from scipy import stats
import math

from skimage.util import img_as_bool, img_as_ubyte
from skimage.morphology import erosion as erode
from skimage.morphology import closing
from skimage.morphology import disk

from scipy.ndimage import label as scipylabel

from skimage.measure import regionprops
import numpy as np
import matplotlib.pyplot as plt

import os
import fnmatch

import ipywidgets as widgets
from ipywidgets import interact

from datetime import datetime 
import pickle

import matplotlib.gridspec as gridspec
import matplotlib as mpl

## For zc mac
#img = mpimg.imread(r'/Users/zackcohen/Dropbox/lily project/090120 sarahs original images/binarized slices/1736 mask only.JPG')
## For zc PC
#img = mpimg.imread(r'C:\Users\zackc\Dropbox\lily project\090120 sarahs original images\binarized slices\1736 mask only.JPG')

In [2]:
def separate_tepals(im):
  # Gunnar wrote this function to seperate each tepal within a slice image

  # Remove scale bar and scale text
  im_c = closing(im, disk(4))

  # Ensure no data is lost in the closing step by enlarging im_c and performing boolean logic with original image to isolate tepals
  im_c_overlap = erode(im_c, disk(7))
  im_bool = img_as_bool(im)
  im_teps = np.invert(np.logical_and(np.invert(im_bool), np.invert(im_c_overlap)))

  # Label tepals
  label_teps, _ = scipylabel(np.invert(im_teps))

  # Isolate tepals
  tepals = []

  for tepal in regionprops(label_teps):
    crop = np.pad(tepal.image, np.round(0.05*len(tepal.image)).astype(int))
    tepals.append(crop)

  # Save tepals
  tepal1 = img_as_ubyte(tepals[0])
  tepal2 = img_as_ubyte(tepals[1])
  tepal3 = img_as_ubyte(tepals[2])

  return tepal1, tepal2, tepal3


def check_tepalStacks(index, mask_OR_photo=0):
  # Gunnar wrote this function to display the slices of each tepal
  fig, ax = plt.subplots(ncols=4, figsize=(24,6))

  if mask_OR_photo == 0:
    ax[0].imshow(slices[index], cmap='gray')
  else:
    ax[0].imshow(tepal_photos[index], cmap='hsv')

  ax[1].imshow(tepals2[index], cmap='gray')
  ax[2].imshow(tepals1[index], cmap='gray')
  ax[3].imshow(tepals3[index], cmap='gray')

  ax[0].set_title(str(files[index]))
  ax[1].set_title('tepals2: Slice ' + str(index))
  ax[2].set_title('tepals1: Slice ' + str(index))
  ax[3].set_title('tepals3: Slice ' + str(index))

  for axi in ax:
    axi.axis('off')
    
    
def switch_tepals(switch_indices, tepal_list1, tepal_list2):
  # Gunnar's function to swap incorrectly assigned tepal slices
    
  # Switch tepals at given indices between tepal_list1 and tepal_list2
  for index in switch_indices:

    # Save tepal images that will be switched to two intermediate variables before overwriting
    temp_tepal1 = tepal_list1[index]
    temp_tepal2 = tepal_list2[index]

    # Switch incorrect tepal for correct tepal in both arrays
    tepal_list1[index] = temp_tepal2
    tepal_list2[index] = temp_tepal1
  
  return tepal_list1, tepal_list2

In [ ]:
# Import images of lily slices, seperate tepals within each slice, organize the slices from each tepal.

dir_bigwhite = r'C:/Users/zackc/Dropbox/lily project/081021 data/tiger lily 2Aug2012/Slices of tiger lily 2Aug2012/'

# Sort images by number
files = []
substring = 'mask only'
for filename in os.listdir(dir_bigwhite):
  if substring in filename:
    files.append(filename)
files.sort()

# Put images into the array called tepal_photos
tepal_names = []
substring2 = 'IMG_'
for name in os.listdir(dir_bigwhite):
  if substring2 in name:
    tepal_names.append(name)
tepal_names.sort()
tepal_photos = []
for k, photo_name in enumerate(tepal_names):
  current_photo = plt.imread(dir_bigwhite + photo_name)
  tepal_photos.append(current_photo)
    
# Put the images into an array
slices = []
for i, slice_name in enumerate(files):
  current_slice = plt.imread(dir_bigwhite + slice_name)   # Read file into current_slice
  if current_slice.ndim > 2:                              # Check dimensionality, and save only 2 dim array
    current_slice = current_slice[:,:,0]
  slices.append(current_slice)                            # Convert to boolean, then append to slices array

# Initialize arrays for tepal slices
tepals1 = []
tepals2 = []
tepals3 = []
for nth, image in enumerate(slices):
  tepal1, tepal2, tepal3 = separate_tepals(slices[nth])   # Separate tepals
  print('Slice ' + str(nth) + ': Completed')
  tepals1.append(True * (tepal1 > 150))    # Append to tepal slice arrays
  tepals2.append(True * (tepal2 > 150))
  tepals3.append(True * (tepal3 > 150))

# Switch indices for tepals1 and tepals2
switch_1and2 = [0, 27]
tepals1, tepals2 = switch_tepals(switch_1and2, tepals1, tepals2)

# Switch indices for tepals2 and tepals3
switch_2and3 = []
tepals2, tepals3 = switch_tepals(switch_2and3, tepals2, tepals3)

interact(check_tepalStacks, index=widgets.IntSlider(value=0, min=0, max=len(slices)-1, step=1), mask_OR_photo=widgets.IntSlider(value=0, min=0, max=1, step=1))


In [ ]:
def corner_detection(image):
    # Check each point on the skeleton. If the point has only 1 neighbor on the skeleton, that point is an endpoint.
    # img = binary 2D array that gives an image of a skeletonized tepal
    
    endpointsX = []
    endpointsY = []
    out = np.nonzero(image) # This is a tuple which contains 2 arrays. The first array contains x-coordinates for points on the skeleton.  The second array contains y-coordinates for points on the skeleton.
    
    # Loop through each point on the skeleton
    for pixelI in range(len(out[0])):
        white_neighbor_pixels = 0
        x_cord = out[0][pixelI]
        y_cord = out[1][pixelI]
        # For the given point on the skeleton, check how many of the 8 adjacent points are also on the skeleton
        for Xchange in [-1, 0, 1]:
            for Ychange in [-1, 0, 1]:
                if image[x_cord + Xchange, y_cord + Ychange] == True:
                    white_neighbor_pixels += 1
        # If only one of the adjacent points is on the skeleton, that point is an endpoint. (In this case, white_neighbor_pixels == 2 because the original point is included too.)
        if white_neighbor_pixels == 2:
            endpointsX.append(x_cord)
            endpointsY.append(y_cord)
    return(endpointsX,endpointsY)

def maximally_seperated_points2(endpointsX, endpointsY, skel):
    # Find the points on a tepal skeleton that are maximally distant from each other, when distance is calculated ALONG THE SKELETON
    # endpointsX = array that contains the x-coordinates for endpoints of the skeleton
    # endpointsY = array that contains the y-coordinates for endpoints of the skeleton.
    # skel = 2dimensional array where pixels on the skeleton are true, everything else is false
    max_distance = 0
    maxP1 = -1
    maxP2 = -1
    max_skel_pixel_list = -1
    # Loop through all pairs of points
    for i in range(len(endpointsX)):
        point1 = (endpointsX[i], endpointsY[i])
        for ii in range(i, len(endpointsX)):
            point2 = (endpointsX[ii], endpointsY[ii])
            # Calculate distance ALONG THE SKELETON between point1 and point2
            skel_pixel_list = skel_pixel_list = skel_to_list(skel, point1, point2)
            distance = len(skel_pixel_list)
            # If point1 and point2 are maximally seperated, keep them
            if distance > max_distance:
                max_distance = np.copy(distance)
                maxP1 = np.copy(point1)
                maxP2 = np.copy(point2)
                max_skel_pixel_list = skel_pixel_list
    skelX, skelY = skel_list_to_XY(max_skel_pixel_list)
    return(skelX, skelY)

def neighbor_pixels(p1,s):
    #p1 = (x,y)
    # s= 2dimensional array where pixels on the skeleton are true, everything else is false
    # return= neighbors= list of neighboring pixels that are white
    neighbors=[]
    for Xchange in [-1,0,1]:
        for Ychange in [-1,0,1]:
            if Xchange==0 and Ychange==0:
                continue
            if s[p1[0]+Xchange,p1[1]+Ychange]==True:
                 neighbors.append((p1[0]+Xchange,p1[1]+Ychange))
    return(neighbors)

def skel_to_list(s, start, end):
    # Given a tepal skeleton, and coordinates for the endpoints, find coordinates for only the skeleton pixels which lie between the endpoints. Do not find branches off the skeleton.
    # To do this, run a "breadth first search".  Mostly written by Gabe Cohen.
    # s = 2dimensional array where pixels on the skeleton are true, everything else is false
    # start = tuple with x and y coordinates for one of the endpoints
    # end = tuple with x and y coordinates for the other endpoint
    # return: l = list of pixels, from end of the skeleton to the start.  Not including branches
    frontier = [start]
    visited = np.zeros_like(s)
    parents = -np.ones((s.shape[0],s.shape[1], 2))
    while len(frontier) != 0:
        node = frontier.pop()
        for neigh in neighbor_pixels(node, s):
            if visited[neigh] == 0 and not neigh in frontier:
                frontier = [neigh] + frontier
                parents[neigh[0],neigh[1],0] = node[0]
                parents[neigh[0],neigh[1],1] = node[1]
        visited[node] = 1
    cur = end
    l = [cur]
    while cur != start:
        l += [tuple(parents[cur[0],cur[1],:])]
        cur = tuple(parents[cur[0],cur[1],:].astype(int))
    return l

def skel_list_to_XY(skel_list):
    # Given a list of tuples (each tuple contains the x,y coordinates for a point on the skeleton), organize the x,y coordinates into seperate lists
    # skel_list = list of tuples. Each tuple contains the x,y coordinates for a point on the skeleton. The tuples are in order from start of the skeleton to the end.  The skeleton doesn't include branches
    # return the Xvalues and Yvalues of those pixels in seperate lists.  Maintains same order of the pixels along the skeleton
    Xvals=[]
    Yvals=[]
    for ABC in skel_list:
        Xvals.append(ABC[0])
        Yvals.append(ABC[1])
    return(Xvals,Yvals)

def find_skeleton(img):
    # Find the x and y coordinates for each point on a tepal slice's skeleton
    # img = 2D boolean array that gives an image of a tepal
    
    skel = skeletonize(img)
    
    # Find all endpoints on the skeleton (including branches)
    endpointsX, endpointsY = corner_detection(skel)
    
    # Find which 2 endpoints are maximally distant
    # Return all the skeleton points which lie between the maximally distant endpoints (remove any branches off the skeleton)
    skelX, skelY = maximally_seperated_points2(endpointsX, endpointsY, skel)
    
    # order the skeleton so that the endpoint nearest x = 0 is first
    if skelX[0]>skelX[-1]:
        skelX1=skelX[::-1]
        skelY1=skelY[::-1]
    else:
        skelX1=skelX
        skelY1=skelY
    return(skelX1,skelY1)

def linear_line(m, x, b):
    return(m*x + b)

def measure_width_1direction(center, increment, slope, intercept, image):
    # Calculate thickness of the tepal along a line.  The line is given by slope and intercept, and passes through center
    # To do this, start at center. Move along the line given by slope and intercept in very small steps.  Once you enter the next pixel, check image == True.  If true, keep taking small steps while the next pixel has image = True
    # center = (x,y) = tuple of initial coordinates for the line
    # increment = -.1 or .1 (step size for moves along the Xaxis.  Might not be large enough to move to the next pixel, so check that pixel has changed)
    # slope, intercept = slope and intercept for the line
    # image = ndarray for tupal image.  Pixels in the tupal are True, otherwise false
    thickness = 0
    pixel_list = []
    current_position = center
    current_pixel = center
    while image[current_pixel] == True:
        # Has the current pixel been visited yet?
        if current_pixel not in pixel_list:
            pixel_list.append(current_pixel)
        # Calculate the next position to move to
        x_update = current_position[0] + increment
        y_update = linear_line(slope, x_update, intercept)
        # If this next position is well-defined, calculate distance between next and current positions.  Replace current position with next position, and current_pixel with next_pixel.
        if x_update > 0 and x_update < np.shape(image)[0] and y_update > 0 and y_update < np.shape(image)[1]:
            thickness += ((current_position[0] - x_update)**2 + (current_position[1] - y_update)**2)**.5
            current_position = (x_update,y_update)
            current_pixel = (int(current_position[0]),int(current_position[1]))
        else:
            break
    return(thickness, pixel_list)

def thickness_in_Ydirection(img, center):
    # Calculate thickness of the tepal along a vertical line through the center
    # img = image of the tuple
    # center = (x,y) = tuple giving the position on the skelton at which to calculate thickness
    
    thickness = 0
    thickness_pixels = []
    
    #count thickness in +Y direction
    current = center
    while img[current] == True:
        thickness += int(img[current])
        thickness_pixels.append(current)
        y_update = current[1] + 1
        current = (center[0], y_update)
    
    #count thickness in -Y direction
    current = center
    while img[current] == True:
        thickness += int(img[current])
        thickness_pixels.append(current)
        y_update = current[1]-1
        current = (center[0], y_update)
    return(thickness, thickness_pixels)

def perpendicular_thicknessV2(img, center, slope):
    # calculate thickness = number of pixels along a line that is perpendicular to the skeleton at a given point
    # img = image of the tuple
    # center = (x,y) = tuple giving the position on the skelton at which to calculate thickness
    # slope = slope of the skeleton at the pixel given by "center"
    if slope is None:  
        # If slope is None, something went wrong at this point.  Return None
        perp_slope = None
        perp_intercept = None
        thickness = None
        thickness_pixels = []
    elif abs(slope) <= 10**-3:
        # If slope = 0, perpendicular is in Y-direction.  No defined slope or intercept. Calculate thickness in Y-direction.
        perp_slope=None
        perp_intercept=None
        thickness, thickness_pixels = thickness_in_Ydirection(img, center)
    elif np.isnan(slope) == True or math.isinf(slope) == True:
        # If slope is infinite, the perpendicular line is horizontal.
        perp_slope = 0
        perp_intercept = center[1]
    else:
        # For all other cases, calculate perpendicular slope and intercept
        perp_slope=-1/slope
        perp_intercept=center[1]-perp_slope*center[0]
    if perp_slope is not None:
        # calculate thickness along the line given by perp_slope and perp_intercept
        thickness1, pixel_list1 = measure_width_1direction(center, -.1, perp_slope, perp_intercept, img) #count thickness in -X direction
        thickness2, pixel_list2 = measure_width_1direction(center, .1, perp_slope, perp_intercept, img) #count thickness in +X direction
        thickness_pixels = pixel_list1 + pixel_list2    
        thickness = thickness1 + thickness2
    return(thickness, thickness_pixels)

def optimize_interval_for_min_distance(img, index_along_skeleton, skelX, skelY):
    # For a given pixel along the skeleton, find the "interval" length that yields the minimum thickness
    # img = original image of the tepal slice
    # the "interval" length is the number of skeleton pixels surrounding the given pixel that will be used to find the slope
    # index_along_skeleton = the given pixel on the skeleton, at which to calculate slope
    # skelX and skelY = X and Y coordinates of the pixels along the skeleton.  The first point on the skeleton is given by (skelX[0],skelY[0])
    
    thicknesses = []
    thickness_pixel_list = []
    intervals = []
    slopes = []
    
    # the maximum interval size to try is either 50, or 2 times the #of pixel between the given point and the end of the skeleton
    max_interval= min([50, index_along_skeleton*2, (len(skelX) - index_along_skeleton)*2])
    
    # Loop through each interval sizes, and calculate thickness for each interval size.  Minimum interval size is 5 pixels
    for intervalI in range(5, max_interval):
        XVALUES = skelX[index_along_skeleton - int(intervalI/2): index_along_skeleton + int(intervalI/2)]
        YVALUES = skelY[index_along_skeleton - int(intervalI/2): index_along_skeleton + int(intervalI/2)]
        # Use linear regression to calculate slope across the given interval
        slope, intercept, r_value, p_value, std_err = stats.linregress(XVALUES, YVALUES)
        # Center = center point of the given interval
        center = (int(skelX[index_along_skeleton]), int(skelY[index_along_skeleton]))
        # Based on the slope, calculate thickness of the tepal at the center point
        thickness, thickness_pixels = perpendicular_thicknessV2(img, center, slope)
        # Keep track of slope, interval length, thickness, and the pixels used to compute that thickness
        intervals.append(intervalI)
        thicknesses.append(thickness)
        thickness_pixel_list.append(thickness_pixels)
        slopes.append(slope)
    # Of all the interval lengths, the one that yields the minimum thickness is kept
    index_for_best = thicknesses.index(min(thicknesses))
    best_interval = intervals[index_for_best]
    best_thickness_pixels = thickness_pixel_list[index_for_best]
    best_slope = slopes[index_for_best]
    return(min(thicknesses), best_thickness_pixels, best_interval, best_slope)

def thickness_along_skel(img, skelX, skelY):
    # for each pixel along the skeleton, calculate the thickness of the tepal
    thickness_list = []
    thickness_pixels_list = []
    interval_list = []
    slope_list = []
    for i in range(len(skelX)):
        if i <= 5 or i >= len(skelX) - 5:
            thickness = None
            thickness_pixels = []
            interval = None
            slope = None
        else:
            thickness, thickness_pixels, interval, slope = optimize_interval_for_min_distance(img, i, skelX, skelY)
        thickness_list.append(thickness)
        thickness_pixels_list.append(thickness_pixels)
        interval_list.append(interval)
        slope_list.append(slope)
    return(thickness_list, thickness_pixels_list, interval_list, slope_list)

def measure_thickness_for_tepal_slice(img):
    # Given an image of a slice of a SINGLE tepal, measure thickness along the length of the tepal
    # img = numpy boolean array which gives an image of a slice from a SINGLE tepal
    Xskel, Yskel = find_skeleton(img)
    thickness_list, thickness_pixels_list, interval_list, slope_list = thickness_along_skel(img, Xskel, Yskel)
    return(thickness_list, thickness_pixels_list, interval_list, slope_list, Xskel, Yskel)

In [ ]:
# For one single tepal, measure thickness along it's length for a given slice.  Repeat this for each slice.

# Choose which tepal you want to analyze. tepal_list = list of images (np boolean arrays) of slices of the given tepal 
tepal_list = tepals1

start_time = datetime.now() 
total_output_dict = {}
for i in range(len(tepal_list)):
    thickness_list, thickness_pixels_list, interval_list, slope_list, Xskel, Yskel = measure_thickness_for_tepal_slice(tepal_list[i])
    total_output_dict[i] = [thickness_list, thickness_pixels_list, interval_list, slope_list, Xskel, Yskel]
    cur_time = datetime.now() 
    print(str(i+1) + '/' + str(len(tepal_list)))
    print('Time elapsed (hh:mm:ss.ms) {}'.format(cur_time - start_time))

In [ ]:
# Save data to a file
# The file loads the variable "save_data".
# The first element of "save_data" is the dict "total_output_dict" (total_output_dict[5] = [thickness_list, thickness_pixels_list, interval_list, slope_list, Xskel, Yskel] for the 5th slice in the given tepal)
# The second element of "save_data" is the list "tepal_list" (tepal_list = list of images (np boolean arrays) of slices of the given tepal)

save_data = [total_output_dict, tepal_list]
outfile = open('100421_tepal1_data','wb')
pickle.dump(save_data, outfile)
outfile.close()
